# 装饰器

本代码基于博文: [《Finally understanding decorators in Python》](https://pouannes.github.io/blog/decorators/)

## 1. 装饰器基础

一个带装饰器的函数看起来是这样的：

In [1]:
def decorator_name(func):
    def f():
        rv = func()
        return rv
    return f

@decorator_name
def func_name():
    pass

在上面这段代码中，`@decorator_name` 就是装饰器。

顾名思义，装饰器是用来装饰东西的。用来装饰什么东西呢？

它是用来装饰**函数**的。从作用上来说，就是把函数捉出来“装饰”一番，然后放回去。

下面通过一些例子说明装饰器的用法和意义。

首先我们写一个 `add` 函数，其中 `y` 的默认值是 10。

In [2]:
def add(x, y = 10):
    return x + y

测试一下这个函数： 

In [3]:
def add(x, y = 10):
    return x + y

print('add(10)', add(10))
print('add(20, 30)', add(20, 30))
print('add("a", "b")', add("a", "b"))

add(10) 20
add(20, 30) 50
add("a", "b") ab


顺便把 `add` 函数的属性打印出来看看：

In [4]:
add.__name__

'add'

In [5]:
add.__module__

'__main__'

In [6]:
add.__defaults__

(10,)

In [7]:
add.__code__.co_varnames  # the variable names of the 'add' fucntion

('x', 'y')

In [8]:
from inspect import getsource
print(getsource(add))

def add(x, y = 10):
    return x + y



接下来，我们用 `time` 模块打印 `add` 函数的运行时间：

In [9]:
from time import time

def add(x, y = 10):
    return x + y

before = time()
print('add(10)', add(10))
after = time()
print('time taken: ', after - before)
defore = time()
print('add(20, 30)', add(20, 30))
after = time()
print('time taken: ', after - before)
before = time()
print('add("a", "b")', add("a", "b"))
after = time()
print('time taken: ', after - before)

add(10) 20
time taken:  0.00016808509826660156
add(20, 30) 50
time taken:  0.00033092498779296875
add("a", "b") ab
time taken:  6.079673767089844e-05


通过阅读上面的代码，我们发现：每次运行，都要先记录一下开始和结束时间，然后 `print` 一下。

为了做这一件事，写了一堆代码。

显然，我的 Python 不可能这么麻烦。重复的代码可以用一个函数装起来，多次利用：

In [10]:
from time import time

def add(x, y = 10):
    before = time()
    rv = x + y
    after = time()
    print('time taken: ', after - before)
    return rv

print('add(10)', add(10))
print('add(20, 30)', add(20, 30))
print('add("a", "b")', add("a", "b"))

time taken:  0.0
add(10) 20
time taken:  9.5367431640625e-07
add(20, 30) 50
time taken:  9.5367431640625e-07
add("a", "b") ab


通过上面这个函数，每次调用 `add` 函数的时候，都会 `print` 函数的运行时间。

到目前为止，这一切都运行的很完美。但如果我们现在有一个 `sub` 函数，然后同样需要打印函数的运行时间呢？

下面是一个 `sub` 函数：

In [11]:
def sub(x, y = 10):
    return x - y

最直接的方法当然就是像上面一样，对 `add` 函数进行改造。

但是如果需要打印时间的不止是 `add` 和 `sub`，还有其他很多函数呢？一个一个地改造函数，就十分麻烦了。

我的 Python 不可能这么麻烦！一定有更简单的方法来实现这个功能。

In [12]:
def add(x, y = 10):
    return x + y

def sub(x, y = 10):
    return x - y

def timer(func, x, y = 10):
    before = time()
    rv = func(x, y)
    after = time()
    print('time taken: ', after - before)
    return rv

print('add(10)', timer(add, 10))
print('sub(10)', timer(sub, 20))

time taken:  0.0
add(10) 20
time taken:  1.1920928955078125e-06
sub(10) 10


上面构造了一个函数 `timer`。

它将 `add`, `sub` 及 `add`, `sub` 的输入作为它的输入，把 `add`, `sub` 的输出作为它的输出。

这就相当于，函数 `timer` 用它自己替代了 `add` 和 `sub` 的功能。`add` 和 `sub` 在这里退化成了函数 `timer` 的参数。

这也不失为一种实现方法，但是这偏离了我们最初**改造函数**的目的。

我们希望经过改造后的函数仍然是一个函数，而非退化成其他函数的参数。

In [13]:
from time import time


def timer(func):
    def f(x, y = 10):
        before = time()
        rv = func(x, y)
        after = time()
        print('time taken: ', after - before)
        return rv
    return f

def add(x, y = 10):
    return x + y

def sub(x, y = 10):
    return x - y

add = timer(add)
sub = timer(sub)

print('add(10)', add(10))
print('add(10, 20)', add(10, 20))
print('sub(20)', sub(20))

time taken:  0.0
add(10) 20
time taken:  0.0
add(10, 20) 30
time taken:  0.0
sub(20) 10


上面这段代码的本质是利用函数 `timer` 对函数 `add` 进行改造。

函数 `timer` 返回的结果是一个新的经过改造的 `add` 函数。

上面这段代码已经完全实现了装饰器的功能。在文章开头的，带 `@` 形式的装饰器，无非是以上过程的简写。

下面尝试用带 `@` 形式写出装饰器 `timer`。为了避免污染变量，我们命名一个新装饰器 `ntimer`。

In [14]:
from time import time


def ntimer(func):
    def f(x, y = 10):
        before = time()
        rv = func(x, y)
        after = time()
        print('time taken: ', after - before)
        return rv
    return f

@ntimer
def add(x, y = 10):
    return x + y

print('add(10)', add(10))

time taken:  0.0
add(10) 20


## 2. 一个小练习

小尝试：

既然学会了写装饰器，我们就来 freestyle 一下。

尝试改造 `new_print` 函数，使每次使用它打印内容时，自动在内容前加上"You just typed: "。

In [15]:
def change_print(func):
    def f(x):
        rv = func('You just typed: ' + x)
        return rv
    return f

@change_print
def new_print(x):
    print(x)

In [16]:
new_print('a')

You just typed: a


## 3. 使用 `*args` 和 `**kwargs` 输入参数和关键字参数。

In [17]:
# *args 和 **kwargs
from time import time

def ntimer(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('time taken: ', after - before)
        return rv
    return f

@ntimer
def add(x, y=10):
    return x + y

@ntimer
def sub(x, y=10):
    return x - y

print('add(10)', add(10))
print('add(20, 30)', add(20, 30))
print('sub(20, 5)', sub(20, 5))

time taken:  0.0
add(10) 20
time taken:  1.1920928955078125e-06
add(20, 30) 50
time taken:  9.5367431640625e-07
sub(20, 5) 15


## 4. 高阶装饰器

还是原来的风格，还是原来的味道。本质是没有变的。只不过在外面多包了一层函数，多给了一个参数罢了。

称之为高阶装饰器实在是太给面子了。


In [18]:
# 高阶装饰器

def ntimes(n):
    def inner(f):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner

In [19]:
@ntimes(3)
def add(x, y):
    print(x + y)
    return x + y

rv = add(1, 2)

3
3
3



把外层函数去掉，取中间部分代码，就是上文中的低阶装饰器。

```python
def inner(f):
    def wrapper(*args, **kwargs):
        for _ in range(n):
            rv = f(*args, **kwargs)
        return rv
    return wrapper
```